In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from datetime import date
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm


/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Create regression data

In [2]:
tfidf_path = '../../data/tfidf_toprev_conlen_fulldata/'

In [3]:
lda_path = '../../data/lda_jsd_toprev_conlen/'

In [4]:
tag_path = '../../data/tag_novelty_full/'

In [5]:
fandom_list = ['harry_potter',
 'dcu',
 'doctor_who_&_related_fandoms',
 'star_wars_all_media_types',
 'arthurian_mythology_&_related_fandoms',
 'supernatural',
 'haikyuu',
 'kuroko_no_basuke',
 'hamilton_miranda',
 'dragon_age_all_media_types',
 'the_walking_dead_&_related_fandoms',
 'buffy_the_vampire_slayer',
 'les_miserables_all_media_types',
 'naruto',
 'tolkien_j_r_r_works_&_related_fandoms',
 'shakespare_william_works',
 'hetalia_axis_powers',
 'attack_on_titan',
 'ms_paint_adventures',
 'homestuck',
 'marvel',
 'bishoujo_senshi_sailor_moon',
 'one_direction',
 'sherlock_holmes_&_related_fandoms']

In [24]:
df_all = []
for i,fandom in enumerate(fandom_list):
    df_tfidf = pd.read_csv(os.path.join(tfidf_path,fandom + '_temporal_tfidf_cos_toprev_conlen.tsv'), sep = '\t')
    del df_tfidf['index']
    df_lda = pd.read_csv(os.path.join(lda_path,fandom + '_temporal_lda_jsd_toprev_full.tsv'), sep = '\t')
    del df_lda['index']
    df_tag = pd.read_csv(os.path.join(tag_path,fandom + '_tag_novelty_conlen_full.tsv'), sep = '\t')
    df = pd.merge(df_tfidf, df_lda, on=['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks', 'Category',\
 'ChapterIndex', 'Chapters' ,'Characters','Comments' ,'CompleteDate',\
 'Fandoms', 'Hits' ,'Kudos', 'Language', 'Notes' ,'PublishDate' ,'Rating',\
 'Relationship' ,'Summary' ,'Title' ,'URL' ,'UpdateDate' ,'Words'], how='inner')
    df = pd.merge(df, df_tag, on=['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks', 'Category',\
 'ChapterIndex', 'Chapters' ,'Characters','Comments' ,'CompleteDate',\
 'Fandoms', 'Hits' ,'Kudos', 'Language', 'Notes' ,'PublishDate' ,'Rating',\
 'Relationship' ,'Summary' ,'Title' ,'URL' ,'UpdateDate' ,'Words'], how='inner')
    
    df['fandom_category'] = fandom
    df['tag_novelty'] = df['tag_novelty'] * 100
    
    df['Kudos'] = df['Kudos']/df['Chapters']
#     df['Kudos'] = np.log(df['Kudos'])
    df['Hits'] = df['Hits']/df['Chapters']
#     df['Hits'] = np.log(df['Hits'])
#     df['Comments'] = np.log(df['Comments'])
    df['Bookmarks'] = df['Bookmarks']/df['Chapters']
#     df['Bookmarks'] = np.log(df['Bookmarks'])

#     df = df.dropna(subset=['Kudos'])
#     df = df[np.isfinite(df['Kudos'])]
#     df['Kudos'] = (df['Kudos'] - df['Kudos'].mean())/df['Kudos'].std(ddof=0)

    df_all.append(df)

In [25]:
df_all = pd.concat(df_all)

In [30]:
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all['Kudos'].fillna(0,inplace=True)

In [65]:
np.percentile(df_all.JSD.tolist(), 95)

0.6582136309616544

In [68]:
s = df_all[df_all.JSD >= 0.65]

In [69]:
len(s)

63426

In [70]:
s = s.sort_values(by = 'Kudos')

In [75]:
s.head()[['JSD', 'Kudos']]

,JSD,Kudos
5263,0.656339,0.0
188,0.659779,0.0
181,0.656665,0.0
179,0.660889,0.0
169,0.664385,0.0


In [74]:
s.head(5).URL.tolist()

['http://archiveofourown.org/works/227180',
 'http://archiveofourown.org/works/3761957/chapters/8353838',
 'http://archiveofourown.org/works/3737240/chapters/8284238',
 'http://archiveofourown.org/works/3737240/chapters/8284232',
 'http://archiveofourown.org/works/184627']